In [1]:
pip install openprompt

Note: you may need to restart the kernel to use updated packages.


In [1]:
import csv

# 初始化字典
trec_dict = {}
q_dict = {}

# 从trec.txt中读取数据
with open('trec.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        query_id = row[0]  # 第一列是查询id
        query_content = row[1]  # 第二列是查询内容
        trec_dict[query_id] = query_content

# 从lmdir.txt中读取数据
with open('lmdir.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        query_id = row[0]  # 第一列是查询id
        doc_content = row[6]  # 第七列是文档内容
        # 计算当前查询id的文档数量
        doc_number = len(q_dict.get(query_id, [])) + 1
        # 在文档内容前添加标识
        doc_content = f'Document: {doc_content}'
        # 将查询id和带标识的文档内容添加到字典中
        key = query_id + '\t' + trec_dict.get(query_id, '')
        if key in q_dict:
            q_dict[key] = q_dict[key] + ' ' + doc_content
        else:
            q_dict[key] = doc_content


# 打印出第一个查询id及其对应的文档内容，以验证代码是否正确
for query_id in list(q_dict.keys())[:1]:
    print("Query :", query_id)
    print("Document Contents:", q_dict[query_id])


Query : 264014	how long is life cycle of flea
Document Contents: Document:  _NUM_  Cancel. A flea can live up to a year, but its general lifespan depends on its living conditions, such as the availability of hosts. Find out how long a flea's life cycle can last with tips from a pet industry specialist in this free video on fleas and pest control.Part of the Video Series: Flea Control.ancel. A flea can live up to a year, but its general lifespan depends on its living conditions, such as the availability of hosts. Find out how long a flea's life cycle can last with tips from a pet industry specialist in this free video on fleas and pest control. Part of the Video Series: Flea Control. Document: The cat flea's primary host is the domestic cat, but it is also the primary flea infesting dogs in most of the world. The cat flea can also maintain its life cycle on other carnivores and on omnivores. Humans can be bitten, though a long-term population of cat fleas cannot be sustained and infest 

In [2]:
import itertools
from openprompt.data_utils import InputExample

input_examples = []

# 使用itertools.combinations来获取所有的两两组合，不会重复
for querya_id_content, queryb_id_content in itertools.combinations(q_dict.keys(), 2):
    # 分割查询ID和查询内容
    querya_id, querya = querya_id_content.split('\t', 1)
    documentsa = q_dict[querya_id_content]
    
    queryb_id, queryb = queryb_id_content.split('\t', 1)
    documentsb = q_dict[queryb_id_content]
    
    t=querya_id+':'+queryb_id
    
    # 构建InputExample
    input_example = InputExample(
        guid= len(input_examples),
        label= None,
        meta= {
            "queryaid": querya_id,
            "querya": querya,
            "documentsa": documentsa,
            "querybid": queryb_id,
            "queryb": queryb,
            "documentsb": documentsb
        },
        text_a= "",
        text_b= "",
        tgt_text= t
    )
    
    # 将InputExample添加到列表中
    input_examples.append(input_example)


D:\pd\Anaconda3\lib\site-packages\transformers\generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [3]:
input_examples[0]

{
  "guid": 0,
  "label": null,
  "meta": {
    "documentsa": "Document:  _NUM_  Cancel. A flea can live up to a year, but its general lifespan depends on its living conditions, such as the availability of hosts. Find out how long a flea's life cycle can last with tips from a pet industry specialist in this free video on fleas and pest control.Part of the Video Series: Flea Control.ancel. A flea can live up to a year, but its general lifespan depends on its living conditions, such as the availability of hosts. Find out how long a flea's life cycle can last with tips from a pet industry specialist in this free video on fleas and pest control. Part of the Video Series: Flea Control. Document: The cat flea's primary host is the domestic cat, but it is also the primary flea infesting dogs in most of the world. The cat flea can also maintain its life cycle on other carnivores and on omnivores. Humans can be bitten, though a long-term population of cat fleas cannot be sustained and infest pe

In [4]:
# class MyInput(object):
#     def __init__(self, guid, querya, documentsa, queryb, documentsb, label=None, meta={}):
#         self.guid = guid
#         self.querya = querya
#         self.documentsa = documentsa
#         self.queryb = queryb
#         self.documentsb = documentsb
#         self.label = label
#         self.meta = meta

#     def __repr__(self):
#         return str(self.__dict__)


In [5]:
# query_pairs = [(querya_id, queryb_id) for querya_id in q_dict for queryb_id in q_dict if querya_id > queryb_id]

In [6]:

# examples = []
# for i, (querya_id, queryb_id) in enumerate(query_pairs):
#     querya = querya_id
#     documentsa = q_dict[querya_id]
#     queryb = queryb_id
#     documentsb = q_dict[queryb_id]
#     examples.append(MyInput(guid=i, querya=querya, documentsa=documentsa, queryb=queryb, documentsb=documentsb))


In [7]:
from openprompt.data_utils import InputExample
classes = [ # There are three docs
    "specific",
    "generic"
]
dataset = input_examples

In [8]:
# from transformers import RobertaTokenizer, RobertaModel
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# model = RobertaModel.from_pretrained('roberta-base')

In [9]:
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm("gpt2", "gpt2-large")
#plm, tokenizer, model_config, WrapperClass = load_plm("albert", "albert-base-v2")


Using pad_token, but it is not set yet.


In [10]:
from openprompt.prompts import ManualTemplate

# placeholder_mapping = {
#     '$Querya': 'Querya',
#     '$Documentsa': 'Documentsa',
#     '$Queryb': 'Queryb',
#     '$Documentsb': 'Documentsb',
# }

template_text = 'Given the following two queries and their respective top k documents, Query "{"meta": "querya"}" , its top k search results: {"meta": "documentsa", "shortenable": True} , the second query "{"meta": "queryb"}", its top k search results: {"meta": "documentsb", "shortenable": True} .It can be inferred that the second query is more {"mask"}'


promptTemplate = ManualTemplate(
    text = template_text,
    tokenizer = tokenizer,
    
)

In [11]:
from openprompt.prompts import ManualVerbalizer
promptVerbalizer = ManualVerbalizer(
    classes = classes,
    label_words = {
        "specific":["good","specific","precise","exact"],
        "generic":["bad","generic","general","common","universal","useless"]
    },
    tokenizer = tokenizer,
)

In [12]:
# model.wrapper

In [13]:
from openprompt import PromptForClassification
promptModel = PromptForClassification(
    template = promptTemplate,
    plm = plm,
    verbalizer = promptVerbalizer,
)

In [ ]:
from openprompt import PromptDataLoader
from openprompt.plms.utils import TokenizerWrapper
data_loader = PromptDataLoader(
    dataset = dataset,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=WrapperClass,
)

tokenizing: 0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (102328 > 1024). Running this sequence through the model will result in indexing errors
tokenizing: 451it [05:01,  1.55it/s]

In [14]:
import numpy as np

def softmax(logits):
    e = np.exp(logits)
    return e / np.sum(e)

logits = np.array([-1.5303, -2.4777])
probabilities = softmax(logits)

print(probabilities)


[0.720592 0.279408]


In [ ]:
from collections import defaultdict
import torch
# 初始化一个字典来存储每个query的总分数
scores = defaultdict(int)

# 对于数据加载器中的每个批次
for batch in data_loader:
    # 获取模型的预测结果
    with torch.no_grad():
        logits = promptModel(batch)
        preds = torch.argmax(logits, dim = -1)
        probs = torch.softmax(logits, dim=-1)
        
        
    for i, prob in enumerate(probs):
        # 获取query的id
        query_id = batch.tgt_text[i].split(':')[1]
        
        print(prob[classes.index('specific')])

        # 根据模型的预测概率给query加分
        scores[query_id] += prob[classes.index('specific')]

#     # 对于批次中的每个预测结果
#     for i, pred in enumerate(preds):
#         # 获取query的id
#         query_id = batch.tgt_text[i].split(':')[1]

#         # 如果预测结果是'specific'，则给这个query加2分
#         # 如果预测结果是'generic'，则给这个query加1分
#         # 这是一个假设的分数分配方式，实际的分数分配方式可能需要根据具体情况进行调整
#         print(classes[pred])
#         if classes[pred] == 'specific':
#             scores[query_id] += 2
#         else:
#             scores[query_id] += 0

# 最后，我们得到了一个字典，其中每个query的id对应其总分数
# 我们可以根据这个字典来进行排序
sorted_queries = sorted(scores.items(), key=lambda item: item[1], reverse=True)


tensor(0.7969)
tensor(0.8614)
tensor(0.7990)
tensor(0.7977)
tensor(0.7978)
tensor(0.8047)
tensor(0.8016)
tensor(0.7058)
tensor(0.8666)
tensor(0.7850)
tensor(0.8892)


In [14]:
# import torch

# promptModel.eval()
# with torch.no_grad():
#     for batch in data_loader:
#         logits = promptModel(batch)
#         print(batch.tgt_text)
#         preds = torch.argmax(logits, dim = -1)
#         #print(preds)
#         print(classes[preds])


In [17]:
query_dict = dict(sorted_queries)

In [18]:
query_dict = {k: v.item() for k, v in query_dict.items()}

In [19]:
query_dict

{'1106007': 30.440275192260742,
 '1103812': 30.05437469482422,
 '405717': 28.594501495361328,
 '1121402': 27.66969108581543,
 '1063750': 27.199600219726562,
 '1113437': 26.629318237304688,
 '1112341': 25.814340591430664,
 '168216': 25.64452362060547,
 '1117099': 24.742576599121094,
 '1114646': 23.524402618408203,
 '962179': 22.975719451904297,
 '855410': 21.431386947631836,
 '1115776': 21.428434371948242,
 '1133167': 21.170124053955078,
 '1124210': 20.189632415771484,
 '833860': 19.813648223876953,
 '87181': 18.525596618652344,
 '183378': 18.007831573486328,
 '47923': 17.20870018005371,
 '19335': 16.283599853515625,
 '156493': 15.801138877868652,
 '915593': 15.003220558166504,
 '87452': 14.023430824279785,
 '182539': 13.44023609161377,
 '451602': 12.61876106262207,
 '1037798': 12.127748489379883,
 '490595': 12.062518119812012,
 '527433': 10.879175186157227,
 '148538': 10.005142211914062,
 '146187': 8.881794929504395,
 '443396': 8.43310832977295,
 '359349': 8.054227828979492,
 '207786':

In [20]:
with open('trecdl19rel.txt', 'r') as f:
    gt = dict(line.strip().split() for line in f)

with open('bertpred.txt', 'r') as f:
    bertpred = dict(line.strip().split() for line in f)

query_scores = list(query_dict.values())
gtscores = [int(gt[key]) for key in query_dict.keys()]
bertscores = [int(bertpred[key]) for key in query_dict.keys()]


In [21]:
# import numpy as np
# pccs = np.corrcoef(scores, gtscores)

In [22]:
import numpy as np
def cal_pccs(X, Y):
    XMean = np.mean(X)
    YMean = np.mean(Y)
    #标准差
    XSD = np.std(X)
    YSD = np.std(Y)
    #z分数
    ZX = (X-XMean)/XSD
    ZY = (Y-YMean)/YSD#相关系数
    r = np.sum(ZX*ZY)/(len(X))
    return(r)


In [23]:
pc1 = cal_pccs(query_scores, gtscores)
pc2 = cal_pccs(bertscores, gtscores)
print(pc1)
print(pc2)

0.10515481453939302
0.20646485309897283


In [24]:
from scipy.stats.stats import kendalltau
k1 = kendalltau(query_scores, gtscores)
k2 = kendalltau(bertscores, gtscores)
print(k1)
print(k2)

KendalltauResult(correlation=0.07334129308490794, pvalue=0.49465994677884506)
KendalltauResult(correlation=0.16931033807650897, pvalue=0.13238651626659328)


In [12]:
# from gensim.summarization import bm25
# import jieba
gpt2:
0.06533198011630328
0.20646485309897283
KendalltauResult(correlation=0.06635640802920242, pvalue=0.5366568517716708)
KendalltauResult(correlation=0.16931033807650897, pvalue=0.13238651626659328)

albert:
0.10515481453939302
0.20646485309897283
KendalltauResult(correlation=0.07334129308490794, pvalue=0.49465994677884506)
KendalltauResult(correlation=0.16931033807650897, pvalue=0.13238651626659328)

roberta-large:
0.12948093733527669
0.2064648530989729
KendalltauResult(correlation=0.0942959482520245, pvalue=0.3799248289061834)
KendalltauResult(correlation=0.16931033807650897, pvalue=0.13238651626659328)




# def test_gensim_bm25():
#     # 给定多个文档
#     corpus = ["5万元资金，该做什么行业",
#               "美增加汽车关税，为何汽车价格不降反升",
#               "汽车销售人员的服务水准非常烂，该怎么解决",
#               "未来房价会跌到什么程度",
#               "十万元能上路的汽车，买什么比较好"]
#     # 对每个文档切词（示例作用  不进行去停用词）
#     corpus_cut = [jieba.lcut(line) for line in corpus]

#     # 生成模型
#     bm25Model = bm25.BM25(corpus_cut)

#     test_query = "你想买汽车吗"  # query
#     test_query_cut = jieba.lcut(test_query)

#     scores = bm25Model.get_scores(test_query_cut)  # 计算相似度得分(与corpus_cut顺序对应)
#     print("scores", scores)
#     # 输出
#     for i, j in zip(scores, corpus):
#         print('分值：{},原句：{}'.format(i, j))
#     print('\n')

